In [2]:
import numpy as np


In [39]:
# This is a code to simulate the interaction between two electron beams in a 1D plasma using an electrostatic PIC code, the length is measured in Debye length
length_of_system = 64

# dt or time_step represents the temporal time step in units of plasma period
dt = 0.1

#Frequencies are measured in units of plasma frequency 
plasma_frequency = 1;

# iteration number
iterations_number = 10

# final time

final_time = (iterations_number)*dt

# number of grid points
grid_points = 256
number_of_beams =  2

# Beam 1
number_of_particles_in_beam1= 10000
drift_velocity_beam1 = 5 #Velocities are measured in units of thermal electron velocity
thermal_velocity_beam1 = 1
charge_mass_relation_beam1 = -1;
perturbation_amplitude_beam_1 = 0;
mode_beam1 = 0;


# Beam 2
number_of_particles_in_beam2= 10000
drift_velocity_beam2 = -5 #Velocities are measured in units of thermal electron velocity
thermal_velocity_beam2 = 1
charge_mass_relation_beam2 = -1; 
perturbation_amplitude_beam_2 = 0;
mode_beam2 = 0;


#Ions in the background
number_of_background_ions = 20000;

#Size of the cell
dx = length_of_system/grid_points;

time=np.arange((iterations_number+1)*dt,dt)




In [14]:
def charge(
    charge_mass_beam1: float,
    charge_mass_beam2: float,
    number_of_background_ions: int,
    number_of_particles_in_beam1: int,
    number_of_particles_in_beam2: int,
    length_of_system: float,
    plasma_frequency: float
    ):
    q_1 = (plasma_frequency**2 * length_of_system) / (number_of_particles_in_beam1 * charge_mass_beam1)
    if charge_mass_beam2 > 0:
        charge_mass_beam2 = -q_1 * (number_of_particles_in_beam1 / number_of_particles_in_beam2)
    elif charge_mass_beam2 < 0:
        q_2 = q_1 * (number_of_particles_in_beam1 / number_of_particles_in_beam2)
    else:
        q_2 = 0
    rho_back = (-q_1 / length_of_system) * (number_of_background_ions)
    return q_1, q_2, rho_back